In [2]:
import numpy as np
import torch
import torch.nn as nn

import texttable as tt

In [3]:
seed = 19241924
torch.manual_seed(seed)

## Loading the data

First, we load the dataset with the helper function `load_compas_data`

In [4]:
from load_compas_data import load_compas_data
x, y, x_sens = load_compas_data()
print(f'Number of samples {x.shape[0]}, number of features {x.shape[1]}')

Looking for file 'compas-scores-two-years.csv' in the current directory...
File found in current directory..

Number of people recidivating within two years
-1    2795
 1    2483
dtype: int64


Features we will be using for classification are: ['intercept', 'age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree'] 

Number of samples 5278, number of features 8


In [5]:
## Selecting sensitive feature
x_sens = x_sens['race']
race_type = {'white': 0, 'black': 1} # TODO check

## Preprocessing the data

Our model is of the form: $Y = \sigma (X \theta + b)$ where $X$ is a matrix of size $N\times F$, $\theta$ of size $F\times1$, $b$ is a scalar value and $\sigma(\cdot)$ is the sigmoid function, i.e., $\sigma(x) = (1+\exp(-x))^{-1}$.

In order to simplify the calculi, we are going to write $X\theta + b$ as $\left[X~1\right] \left[\theta^T b\right]^T$. 

That is, we are going to embed the bias value into the set of parameters $\theta$ and add a column of ones to $X$. By calling $X=\left[X~1\right]$ and $\theta=\left[\theta^T b\right]^T$ our model is written as $Y=\sigma(X\theta)$.

In [6]:
x = np.c_[x, np.ones((x.shape[0],1))]

And we are also going to transform $Y$ such that it contains 0s and 1s instead of 1s and -1s.

In [7]:
y = (y + 1)/2
print(y[:20])

[1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.]


As we are going to work with Pytorch we need to transform the data to the right data type. In this case, we have to transform every numpy object to Pytorch's tensors.

In [8]:
x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)
x_sens = torch.tensor(x_sens, dtype=torch.uint8)

type(x)

torch.Tensor

And we are going to split the data into train/test datasets

In [9]:
train_size = int(x.size(0) * 0.8)
random_perm = torch.randperm(x.size(0))
train_idx, test_idx = random_perm[:train_size], random_perm[train_size:]

x_train, x_test = x[train_idx], x[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
x_sens_train, x_sens_test = x_sens[train_idx], x_sens[test_idx]

## Some measures

We are going to define some measures that we will use throughtout the script

In [10]:
def demographic_parity(y, mask):
    #TODO
    return 0 #TODO

In [11]:
def equal_opportunity(y_pred, y, mask):
    #TODO
    return 0 #TODO

In [12]:
def num_positive(y, mask):
    y_cond = y * mask.float()
    return y_cond.mean()

In [13]:
def accuracy(y_pred, y, mask):
    y_cond = mask.float() * ( y * y_pred + (1 - y) * (1 - y_pred))
    return y_cond.sum() / mask.sum()

In [14]:
to_percentage = lambda l: [f'{float(y) * 100.:.2f}%' for y in l]

for name, [x, y, x_sens] in zip(['Train', 'Test'], [[x_train, y_train, x_sens_train], [x_test, y_test, x_sens_test]]):
    table = tt.Texttable()
    table.header([''] + list(race_type.keys()))

    
    table.add_row(['positive class'] + to_percentage([num_positive(y, x_sens == v) for v in race_type.values()]))
    table.add_row(['negative class'] + to_percentage([num_positive(1 - y, x_sens == v) for v in race_type.values()]))
    #table.add_row(['Benefits demographic parity'] # + TODO

    print(f'{name}:')
    print(table.draw())
    
    #print(f'Demographic Disparity = {TODO}')

Train:
+----------------+--------+--------+
|                | white  | black  |
+================+========+========+
| positive class | 31.79% | 15.47% |
+----------------+--------+--------+
| negative class | 28.49% | 24.25% |
+----------------+--------+--------+
Test:
+----------------+--------+--------+
|                | white  | black  |
+================+========+========+
| positive class | 30.21% | 16.00% |
+----------------+--------+--------+
| negative class | 29.45% | 24.34% |
+----------------+--------+--------+


## The model

And let's define our logistic regression model. In Pytorch this is done by creating a subclass of `torch.nn.Module` which define its parameters at `__init__` and acts on the input through the function `forward`. We will initialize our parameters randomly using a $\mathcal{N}(0, 1)$ distribution.

In [15]:
class LogisticRegression(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(size, 1))
        
    def forward(self, x):
        return torch.sigmoid(torch.matmul(x, self.theta)).flatten()

And let's declare some helper functions to see all the info we care about or model

In [16]:
def confusion_matrix(y_pred, y, mask=None):
    size = y.size(0) if mask is None else mask.sum().item()
    sum_and_int = lambda x: x.sum().long().item()
    to_percentage = lambda l: [f'{float(y)* 100. / size :.2f}%' for y in l]
    
    y_pred_binary = (y_pred > 0.5).float()
    if mask is None:
        mask = torch.ones_like(y_pred)
    mask = mask.float()
    
    true_positives = sum_and_int(y * y_pred_binary * mask)
    false_positives = sum_and_int((1 - y) * y_pred_binary * mask)
    
    true_negatives = sum_and_int((1 - y) * (1 - y_pred_binary) * mask)
    false_negatives = sum_and_int(y * (1 - y_pred_binary) * mask)
    
    total_positives = true_positives + false_negatives
    total_negatives = true_negatives + false_positives
    total = total_positives + total_negatives
    
    # Show the confusion matrix
    table = tt.Texttable()
    table.header(['Pred/Real', 'Positive', 'Negative', ''])
    table.add_row(['Positive'] + to_percentage([true_positives, false_positives, true_positives + false_positives]))
    table.add_row(['Negative'] + to_percentage([false_negatives, true_negatives, false_negatives + true_negatives]))
    table.add_row([''] + to_percentage([total_positives, total_negatives, total]))
    
    return table

In [17]:
def measures_table(y_pred, y, x_sens):
    to_percentage = lambda l: [f'{float(y) * 100.:.2f}%' for y in l]
    y_pred_binary = (y_pred > 0.5).float()
    
    table = tt.Texttable()
    table.header([''] + list(race_type.keys()))
    
    
    table.add_row(['accuracy'] + to_percentage([accuracy(y_pred_binary, y, x_sens == v) for v in race_type.values()]))
    table.add_row(['positive class'] + to_percentage([num_positive(y_pred_binary, x_sens == v) for v in race_type.values()]))
    table.add_row(['negative class'] + to_percentage([num_positive(1 - y_pred_binary, x_sens == v) for v in race_type.values()]))
    #table.add_row(['benefit demographic parity'] + TODO
    #table.add_row(['benefit equal opportunity'] + TODO
    
    
    return table

In [18]:
@torch.no_grad()  # To avoid calculating any gradients for the parameters
def print_model_info(model, test=True, samples=20):
    if test:
        print('### Test ###')
        x, y, x_sens = x_train, y_train, x_sens_train
    else:
        print('### Train ###')
        x, y, x_sens = x_test, y_test, x_sens_test
        
    theta = model.theta.flatten()[:-1]
    print(f'Theta:            {theta.tolist()}')
    
    bias = model.theta.flatten()[-1]
    print(f'Bias:             {bias.tolist()}')
    
    y_pred = model(x)
    y_pred_binary = (y_pred > 0.5).float()
    print()
    print(f'Predictions:      {[round(s, 1) for s in y_pred[:samples].tolist()]}')
    print(f'Binarized preds:  {y_pred_binary[:samples].tolist()}')
    print(f'Targets:          {y[:samples].flatten().tolist()}')

    # Build the confusion matrix
    table = confusion_matrix(y_pred, y)    
    print('Confusion matrix:')
    print(table.draw())
    
    for k, v in race_type.items():
        table = confusion_matrix(y_pred, y, x_sens == v)
        print(f'Confusion matrix for race={k}:')
        print(table.draw())
        
    # Print different measures
    table = measures_table(y_pred, y, x_sens)
    print('Fairness measures')
    print(table.draw())
    print()
    print(f'Demographic Disparity = #TODO')
    print(f'Equal Opportinity = #TODO')
    
    print()

## Gradient descent

Now let's create the gradient descent method, that is, update our parameters on the opossite direction of the gradient

In [19]:
@torch.no_grad()  # This is to let Pytorch modify our parameters
def gradient_descent(model, learning_rate):
    model.theta -= learning_rate * model.theta.grad

## Training routine

In [20]:
def train(model, loss_function, x, y, epochs, learning_rate, print_freq=100, **kwargs):
    for i in range(epochs):
        model.zero_grad()  # sets all the gradients back to 0
    
        y_pred = model(x)
        loss = loss_function(y_pred, y, **kwargs)
        if i == 0 or (i+1) % print_freq == 0:
            print(f'Epoch {i+1}: loss {loss.item()}')
            
        loss.backward()  # calculates the gradient of all the parameters
        gradient_descent(model, learning_rate)

Now we can train our model and see if something has changed

## Maximizing efficiency approach

We are going to take the usual approach where we don't consider any fairness constrains and hence we only care about minimizing our loss function

As a loss function we are going to use the average binary cross entropy loss, that is $\frac{1}{N}\sum_{n=1}^N \left[ y_n\log \hat{y}_n + (1-y_n)\log (1-\hat{y}_n) \right]$

In [21]:
my_model_me = LogisticRegression(x.size(1))
my_loss_function = nn.functional.binary_cross_entropy  # cross entropy loss

train(my_model_me, my_loss_function, x_train, y_train, epochs=1000, learning_rate=0.05)

Epoch 1: loss 0.8671475648880005
Epoch 100: loss 0.6692675352096558
Epoch 200: loss 0.6501793265342712
Epoch 300: loss 0.6396610140800476
Epoch 400: loss 0.633036196231842
Epoch 500: loss 0.6286677122116089
Epoch 600: loss 0.6257181167602539
Epoch 700: loss 0.6237037181854248
Epoch 800: loss 0.6223220229148865
Epoch 900: loss 0.621358335018158
Epoch 1000: loss 0.6206861734390259


In [22]:
print_model_info(my_model_me, test=False)

### Train ###
Theta:            [-0.26064425706863403, 0.15851888060569763, -0.33682048320770264, 0.7693789601325989, -0.27491647005081177, 0.44202113151550293, 0.7048741579055786, -0.24221782386302948]
Bias:             -0.18691441416740417

Predictions:      [0.4, 0.3, 0.6, 0.6, 0.5, 0.7, 0.2, 0.9, 0.5, 0.3, 0.5, 0.6, 0.8, 0.5, 0.3, 0.8, 0.6, 0.4, 0.4, 0.4]
Binarized preds:  [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
Targets:          [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
Confusion matrix:
+-----------+----------+----------+---------+
| Pred/Real | Positive | Negative |         |
+===========+==========+==========+=========+
| Positive  | 26.04%   | 13.07%   | 39.11%  |
+-----------+----------+----------+---------+
| Negative  | 20.17%   | 40.72%   | 60.89%  |
+-----------+----------+----------+---------+
|           | 46.21%   | 53.79%   | 100.00% |
+-----------+-

In [23]:
print_model_info(my_model_me, test=True)


### Test ###
Theta:            [-0.26064425706863403, 0.15851888060569763, -0.33682048320770264, 0.7693789601325989, -0.27491647005081177, 0.44202113151550293, 0.7048741579055786, -0.24221782386302948]
Bias:             -0.18691441416740417

Predictions:      [0.6, 0.5, 0.3, 0.3, 0.7, 0.4, 0.5, 0.6, 0.3, 0.4, 0.3, 0.9, 0.4, 0.6, 0.3, 0.4, 0.4, 0.5, 0.9, 0.4]
Binarized preds:  [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Targets:          [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
Confusion matrix:
+-----------+----------+----------+---------+
| Pred/Real | Positive | Negative |         |
+===========+==========+==========+=========+
| Positive  | 27.12%   | 12.96%   | 40.08%  |
+-----------+----------+----------+---------+
| Negative  | 20.13%   | 39.79%   | 59.92%  |
+-----------+----------+----------+---------+
|           | 47.25%   | 52.75%   | 100.00% |
+-----------+--

## Demographic parity approach

## Equal opportunity approach

## Summary

In [24]:
print('Maximum efficiency model:')
print(measures_table(my_model_me(x_test), y_test, x_sens_test).draw())

#print('Demographic parity model:')
#print(measures_table(my_model_dp(x_test), y_test, x_sens_test).draw())

#print('Equal opportunity model:')
#print(measures_table(my_model_eo(x_test), y_test, x_sens_test).draw())

Maximum efficiency model:
+----------------+--------+--------+
|                | white  | black  |
+================+========+========+
| accuracy       | 67.62% | 65.49% |
+----------------+--------+--------+
| positive class | 32.67% | 6.44%  |
+----------------+--------+--------+
| negative class | 26.99% | 33.90% |
+----------------+--------+--------+
